## Ingestion de la carpeta JSON multi-linhas "movie_languages"

#### Paso 0 - Actualización de variables y funciones y validaciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#Chama a função para validar se o arquivo a ser processado existe
valida_arquivo(bronze_folder_path, v_file_date, "movie_language/movie_languages_1.json")

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
#%fs
#ls dbfs:/mnt/moviehistoryadilmor/bronze/movie_language/

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
#Definindo o schema na variavel xxxx_schema
movies_languages_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", StringType(), True)
])

In [0]:
movies_languages_df = spark.read \
               .schema(movies_languages_schema) \
               .option("multiLine", True) \
               .json(f"{bronze_folder_path}/{v_file_date}/movie_language/movie_languages_*.json")

In [0]:
#display(movies_languages_df)
#movies_languages_df.count()

#### Paso 2 - Renombrar el nombre de las columnas,  añadir "ingestion_date" y "enviroment" Y dropar las columnas no deseadas

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col

In [0]:
movies_languages_final_df = add_ingestion_date(movies_languages_df) \
                    .withColumnsRenamed({"movieId": "movie_id", 
                                         "languageId": "language_id"}) \
                    .drop(col("languageRoleId")) \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
#display(movies_languages_final_df)

#### Paso 3 - Escribir datos en el datalake en formato parquet

In [0]:
#Elimina partição se ela já existir
#drop_partition_if_exists(movies_languages_final_df, "movie_silver", "movies_languages", "file_date")

In [0]:
#movies_languages_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_languages/")
#movies_languages_final_df.write.mode("append") \
#                                .partitionBy("file_date") \
#                                .format("parquet") \
#                                .saveAsTable("movie_silver.movies_languages")

#Caso a tabela não existir, cria a tabela com os dados, senão efetua um merge dos dados na tabela
merge_delta_lake(movies_languages_final_df, "movie_silver", "movies_languages", silver_folder_path, "tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id", "file_date")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/movies_languages

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_languages"))
#(spark.read.parquet("/mnt/moviehistoryadilmor/silver/movies_languages")).count()
#spark.sql("select * from movie_silver.movies_languages").display()
display(spark.sql("SELECT file_date, \
                          COUNT(1) \
                   FROM movie_silver.movies_languages \
                   GROUP BY file_date"
                   )
        )

In [0]:
dbutils.notebook.exit("Success")